 1. Run Grover's algorithm using different iteration numbers to solve
3sat_mystery(a,b,c). What happens when we use only one iteration? Zero
iterations? More than the recommended number?

In [ ]:
#we get the same answer but with less certainty
#result appears to be random
#we get the same answer but with far less certainty

2. Switch variables b and c in the 3sat_mystery(a,b,c) function to create a new
mystery function. What result do you expect? Verify that you get this result
classically, and that Grover's algorithm works for this new function to reproduce
the classic result.

In [ ]:
#result should be (1,0,1) instead of (1,1,0) because we switched the last two parameters, b and c


3. Challenge problem: modify the implementation of 3sat_mystery(a,b,c) to
use at least one fewer temporary qubits. Verify your modification still produces
the correct result. Continue modifying the code to use as few temporary qubits as
possible while producing the correct result. What is the smallest number you can
possibly use? The fewer qubits used, the smaller the number of qubits in the
quantum computer necessary to solve the problem.

In [ ]:
#TODO

. 2SAT Grover:
1. Implement the following 2SAT function classically, where each clause
contains the OR of two variables, and the clauses are combined with
an AND: . Verify your function returns
true only for the case of =True, =False. How many inputs did you
need to check classically?
2. Implement your function on a quantum computer using multi-qubit
AND and multi-qubit OR.
3. Modify the code in this chapter to execute Grover's algorithm to find
the input for this 2SAT function that satisfies it. How many iterations
did you need? What happens if you increase or decrease the number of
iterations?
4. Challenge problem: modify the code to run on IBM QX hardware. You
will need to make sure to use no more than the number of qubits
available on the current hardware you have access to, to make sure the
program is of the appropriate length, and to take care that, on IBM
hardware, it is in general possible to compute a CNOT gate between
two qubits only for some pairs of qubits.

In [4]:
import itertools
def _2sat_mystery_classical(a,b):
    return int((a or b) and (not a or not b) and (a or not b))

for combo in itertools.product([0,1], repeat=2):
    print(combo, '->', _2sat_mystery_classical(combo[0], combo[1]))
#we had to check 2^2 combinations of inputs


(0, 0) -> 0
(0, 1) -> 0
(1, 0) -> 1
(1, 1) -> 0


In [14]:
from qiskit import IBMQ
from qiskit import Aer
import qiskit
import matplotlib
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from  qiskit.tools.visualization  import circuit_drawer
from qiskit.extensions.standard import ccx
IBMQ.enable_account('407e9bae3b7dcde32639a7dcedf05ae6d7a0bed5c1a328e149afd88e5a6584d3776bfa94fdbcad4de27402abf2ed41b183ba3e6877fd64b80917cc06c4e820ae','https://api.quantum-computing.ibm.com/api/Hubs/ibm-q/Groups/open/Projects/main')# To store your credentials locally you can run:


def quantumand_2(qr,qc,w,x,t1):
    qc.ccx(qr[w],qr[x],qr[t1])
    return t1

def quantumor_2(qr,qc,w,x,t1):
    qc.x(qr[w])
    qc.x(qr[x])
    qc.x(qr[t1])
    qc.ccx(qr[w],qr[x],qr[t1])
    qc.x(qr[w])
    qc.x(qr[x])
    return t1

def quantumor_2_reverse(qr,qc,w,x,t1):
    qc.x(qr[x])
    qc.x(qr[w])
    qc.ccx(qr[w],qr[x],qr[t1])
    qc.x(qr[t1])
    qc.x(qr[x])
    qc.x(qr[w])
    return t1
def setup_or_teardown_logic(qr,qc,is_a,is_b): 
    """    
    is_a,is_b,and : False indicates the variable should be negated, True left as is.
    Negation is done with the X gate.

    """    
    if not is_a:
        qc.x(qr[0])
    if not is_b:
        qc.x(qr[1])
def _2sat_mystery_2(qr,qc):
    #a or b
    setup_or_teardown_logic(qr,qc,True, True)
    first_clause = quantumor_2(qr,qc,0,1,2)
    setup_or_teardown_logic(qr,qc,True, True)
    
    #not a or not b
    setup_or_teardown_logic(qr,qc,False,False)
    second_clause = quantumor_2(qr,qc,0,1,3)
    setup_or_teardown_logic(qr,qc,False,False)
    

    #a or not b
    setup_or_teardown_logic(qr,qc,True,False)
    third_clause= quantumor_2(qr,qc,0,1,4)
    setup_or_teardown_logic(qr,qc,True,False)
    
    intermediate_and_1 = quantumand_2(qr,qc,first_clause,second_clause,5)
    final = quantumand_2(qr,qc, third_clause, intermediate_and_1,6)

import time
from qiskit.tools.visualization import plot_histogram
def try_input_combination(input_combination,shots=1):
    
    backend = IBMQ.get_backend('ibmq_qasm_simulator') # remote simulator
    qr = QuantumRegister(7)
    cr = ClassicalRegister(7)
    qc = QuantumCircuit(qr,cr)
    # setting up the input
    for i in range(2):
        if input_combination[i]:
            qc.x(qr[i])
    # calling the function on that input
    _2sat_mystery_2(qr,qc)
    # measuring every qubit as we will want to verify reversibility 
    for i in range(7):
        qc.measure(qr[i],cr[i])

    # Executing the job on IBM QX
    job_exp = qiskit.execute(qc, backend=backend,shots=shots)
    result = job_exp.result()
    final=result.get_counts(qc)
    if not len(final)==1:
        print(input_combination,final)
    else:
        # note that due to IBM's choice the result returned is in opposite order with last register coming first 
        # and the first register coming last. For clarity we reverse the output so the first register is first
        # and the last register is last.

        result_in_order=list(final.keys())[0][::-1]
        print(input_combination,'->',result_in_order[-1],'(measured bits: '+result_in_order+')')  
        
import itertools
for combo in itertools.product([0,1],repeat=2):
    try_input_combination(combo)

(0, 0) -> 0 (measured bits: 0001100)
(0, 1) -> 0 (measured bits: 0111010)
(1, 0) -> 1 (measured bits: 1011111)
(1, 1) -> 0 (measured bits: 1110100)


In [20]:
def grovers_algorithm(checker,num_inputs,num_registers,num_iterations=None):
    if num_iterations == None:
        from math import floor,sqrt
        iterations=floor(sqrt(2**num_inputs))
    else:
        iterations=num_iterations
    print("Running Grover's algorithm for %d iterations"%iterations)
    qr = QuantumRegister(num_registers)
    cr = ClassicalRegister(num_registers)
    qc = QuantumCircuit(qr,cr)
    # Configuring the input
    for i in range(num_inputs):
        qc.h(qr[i])
    # Setting up the output of the checker function
    qc.x(qr[num_registers-1])
    qc.h(qr[num_registers-1])
    
    # Do the Grovers steps
    for it in range(iterations):
        checker(qr,qc)
        mover(qr,qc,num_inputs)
    # Measure the inputs
    for j in range(num_inputs):
        qc.measure(qr[j], cr[j])
    return cr,qr,qc

    
def control_Z(qr,qc,num_inputs):
    if num_inputs not in [2,3]:
        raise Exception("currently only supports 2 or 3 inputs")
    if num_inputs==2:
        qc.h(qr[1])
        qc.cx(qr[0],qr[1])
        qc.h(qr[1])
    elif num_inputs==3:
        qc.h(qr[2])
        qc.ccx(qr[0],qr[1],qr[2])    
        qc.h(qr[2])
    
def mover(qr,qc,num_inputs):
    if num_inputs not in [2,3]:
        raise Exception("currently only supports 2 or 3 inputs")
    for i in range(num_inputs):
        qc.h(qr[i])
    for i in range(num_inputs): # D matrix
        qc.x(qr[i]) 
    control_Z(qr,qc,num_inputs)
    for i in range(num_inputs):
        qc.x(qr[i])
    for i in range(num_inputs):
        qc.h(qr[i])
        
def run_2sat_mystery_grover_ibm_sim(num_iterations=None,backend=None):
    if not backend:
        backend=IBMQ.get_backend('ibmq_qasm_simulator')
    import time
    from qiskit.tools.visualization import plot_histogram
    shots=50
    cr,qr,qc = grovers_algorithm(_2sat_mystery_2,2,7,num_iterations=num_iterations)
    job_exp = qiskit.execute(qc, backend=backend,shots=shots)
    result = job_exp.result()
    final=result.get_counts(qc)
    print(final)
    return final
for i in range(0,7):
    final = run_2sat_mystery_grover_ibm_sim(num_iterations=i)
    plot_histogram(final)
#two iterations seems best, but we come back around with five iterations

Running Grover's algorithm for 0 iterations
{'0000011': 9, '0000010': 13, '0000000': 14, '0000001': 14}
Running Grover's algorithm for 1 iterations
{'0000011': 12, '0000010': 13, '0000000': 13, '0000001': 12}
Running Grover's algorithm for 2 iterations
{'0000011': 9, '0000010': 13, '0000000': 4, '0000001': 24}
Running Grover's algorithm for 3 iterations
{'0000011': 14, '0000010': 12, '0000000': 17, '0000001': 7}
Running Grover's algorithm for 4 iterations
{'0000011': 8, '0000010': 14, '0000000': 11, '0000001': 17}
Running Grover's algorithm for 5 iterations
{'0000011': 8, '0000010': 6, '0000000': 12, '0000001': 24}
Running Grover's algorithm for 6 iterations
{'0000011': 10, '0000010': 21, '0000000': 11, '0000001': 8}


In [ ]:
#TODO

5. Apply the mover and checker functions once more to the output by manually
calculating their results:

In [ ]:
#mover(checker(previous_result)) = .1855(|000>+|001>+|010>+|011>+|100>+|101>+|111>+)+ .9923|110> 